In [35]:
pip install scikit-optimize scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [56]:
import warnings

warnings.filterwarnings("ignore")

## Import necessary libraries

In [1]:
import math
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import load_iris
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, make_scorer


In [2]:
train = pd.read_csv(r'C:\Users\Rawan\Desktop\term 7\machine learning\Assignment 3_attachments\Assignment 3_attachments\train_Data.csv', index_col= 0)
test = pd.read_csv(r'C:\Users\Rawan\Desktop\term 7\machine learning\Assignment 3_attachments\Assignment 3_attachments\test_Data.csv', index_col= 0)
val = pd.read_csv(r'C:\Users\Rawan\Desktop\term 7\machine learning\Assignment 3_attachments\Assignment 3_attachments\val_Data.csv', index_col= 0)


In [3]:
train.head()

,AST,Cholesterol,eyesight(right),HDL,triglyceride,eyesight(left),height(cm),Urine protein,smoking
17406,24.0,0.357143,0.368421,-1.076028,0.990459,0.000746,-0.031118,0.0,1
61303,34.0,0.553571,0.578947,1.543598,-1.062704,0.000746,-0.599977,0.0,1
71781,17.0,0.440476,0.315789,0.452087,0.285174,-0.320325,-0.599977,0.0,1
67014,38.0,0.327381,0.473684,0.379319,-1.062704,-0.320325,-1.168836,0.0,0
73322,25.0,0.351190,0.473684,-0.712191,-0.263381,-0.962466,-1.168836,0.0,0


## splitting dataset into x and y

In [7]:
y_train=train['smoking']
x_train=train.drop(columns=['smoking'])

In [8]:
y_test=test['smoking']
x_test=test.drop(columns=['smoking'])

In [9]:
y_val=val['smoking']
x_val=val.drop(columns=['smoking'])

In [10]:
x_train.head()

,Cholesterol,LDL,eyesight(right),HDL,triglyceride,eyesight(left),height(cm),Urine protein,Total Cholesterol,Cholesterol Ratio
17406,0.357143,0.361842,0.368421,-1.076028,0.990459,0.000746,-0.031118,0.0,-0.833333,0.124537
61303,0.553571,0.513158,0.578947,1.543598,-1.062704,0.000746,-0.599977,0.0,0.571429,-0.631807
71781,0.440476,0.375000,0.315789,0.452087,0.285174,-0.320325,-0.599977,0.0,-0.285714,-0.606227
67014,0.327381,0.375000,0.473684,0.379319,-1.062704,-0.320325,-1.168836,0.0,-0.309524,-0.581288
73322,0.351190,0.434211,0.473684,-0.712191,-0.263381,-0.962466,-1.168836,0.0,-0.452381,0.117167


## Ensemble Methods

## 1)Boosting (AdaBoost)

In [42]:
def compute_error(y, y_pred, w_i):
    y = np.array(y)  
    y_pred = np.array(y_pred) 
    return np.sum(w_i * (y != y_pred)) / np.sum(w_i)

def compute_alpha(error):
    return 0.5 * np.log((1 - error) / error)

def update_weights(w_i, alpha, y, y_pred):
    return w_i * np.exp(-alpha * y * y_pred)

class AdaBoost:
    def __init__(self, model_params=None, n=100, base_model=None):  
        self.alphas = []
        self.models = []
        self.n = n
        self.base_model = base_model
        self.model_params = model_params if model_params is not None else {}
        self.training_errors = []
        self.prediction_errors = []

    def get_params(self, deep=True):
        return {'n': self.n, 'model_params': self.model_params,  'base_model': self.base_model}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit(self, x, y):
        self.alphas = []
        self.training_errors = []
        w_i = np.ones(len(y)) / len(y)

        for m in range(self.n):
            if self.base_model is None:
                model = DecisionTreeClassifier(max_depth=1)
            else:
                model = self.base_model.set_params(**self.model_params)


            model.fit(x, y, sample_weight=w_i)
            y_pred = model.predict(x)

            error_m = compute_error(y, y_pred, w_i)
            alpha_m = compute_alpha(error_m)

            w_i = update_weights(w_i, alpha_m, y, y_pred)

            self.models.append(model)
            self.alphas.append(alpha_m)
            self.training_errors.append(error_m)

    def predict(self, x):
        weak_preds = np.zeros((len(x), self.n))

        for model in range(self.n):
            y_pred_model = self.models[model].predict(x) * self.alphas[model]
            weak_preds[:, model] = y_pred_model

        y_pred = np.sign(weak_preds.sum(axis=1))
        return y_pred.astype(int)

    def error_rates(self, X, y):
        self.prediction_errors = []

        for model in range(self.n):
            y_pred_model = self.models[model].predict(X)
            error_model = compute_error(y, y_pred_model, np.ones(len(y)))
            self.prediction_errors.append(error_model)
    def score(self, X, y):
        # Assuming you want to use accuracy as the scoring metric
        y_pred = self.predict(X)
        accuracy = accuracy_score(y, y_pred)
        return accuracy
adaboost = AdaBoost()
adaboost.fit(x_train, y_train)
y_pred = adaboost.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7053874168027126


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, make_scorer



param_grid = { 
    'base_model': [None,DecisionTreeClassifier(), LogisticRegression(), KNeighborsClassifier(), LinearRegression()],
    'n': [None,10,20,30,80, 100, 120],  # Number of weak learners
    'model_params': [
        {'max_depth': [1,2,3], 'criterion': ['gini', 'entropy']},  # Parameters for DecisionTreeClassifier
        {},  # Parameters for LogisticRegression
        {'n_neighbors': list(range(1, 20))},  # Parameters for KNeighborsClassifier
        {}
    ]
}
adaboost = AdaBoost()
scorer = make_scorer(accuracy_score)
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, scoring=scorer, cv=5)#k folds
grid_search.fit(x_train, y_train)
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)


best_model = grid_search.best_estimator_
val_accuracy = best_model.score(x_val, y_val)
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))

# Test the best model on the test set
test_accuracy = best_model.score(x_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

Best Parameters:  {'base_model': None, 'model_params': {'max_depth': [1, 2, 3], 'criterion': ['gini', 'entropy']}, 'n': 10}
Best Accuracy:  0.7015761263991751
Validation Accuracy: 70.38%
Test Accuracy: 70.54%


In [11]:
param_dist = { 
    'base_model': [None,DecisionTreeClassifier(), LogisticRegression(), KNeighborsClassifier(), LinearRegression()],
    'n': [None,10,20,30,80, 100, 120],  # Number of weak learners
    'model_params': [
        {'max_depth': [1,2,3], 'criterion': ['gini', 'entropy']},  # Parameters for DecisionTreeClassifier
        {},  # Parameters for LogisticRegression
        {'n_neighbors': list(range(1, 20))},  # Parameters for KNeighborsClassifier
        {}
    ]
}
adaboost = AdaBoost()
scorer = make_scorer(accuracy_score)
random_search = RandomizedSearchCV(estimator=adaboost, param_distributions=param_dist, n_iter=18, scoring=scorer, cv=5, random_state=42)
random_search.fit(x_train, y_train)
print("Best Parameters: ", random_search.best_params_)
print("Best Accuracy: ", random_search.best_score_)


best_model = random_search.best_estimator_
val_accuracy = best_model.score(x_val, y_val)
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))

# Test the best model on the test set
test_accuracy = best_model.score(x_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

Best Parameters:  {'n': 100, 'model_params': {}, 'base_model': None}
Best Accuracy:  0.7015761263991751
Validation Accuracy: 70.38%
Test Accuracy: 70.54%


## 2)Bagging

In [67]:
class Bagger:
    
    def __init__(self, seed=None,  model_params=None, M=100, base_model=None):
        self.alphas = []
        self.models = []
        self.M = M
        self.base_model = base_model
        self.model_params = model_params if model_params is not None else {}
        self.seed = seed
        self.training_errors = []
        self.prediction_errors = []
        
    def get_params(self, deep=True):
        return {'M': self.M, 'model_params': self.model_params,  'base_model': self.base_model}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    
    #it samples the training data with replacement 
    #and fits a base model (DecisionTreeClassifier by default) to this sampled data.
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.N, self.D = x_train.shape# .N -> num of samples , .d -> num of features
        self.y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
        np.random.seed(self.seed)
        for m in range(self.M):
            sample = np.random.choice(np.arange(self.N), size=self.N, replace=True)
            sample = sample.astype(int)
            if isinstance(x_train, pd.DataFrame):
                x_train_m = x_train.iloc[sample]
            else:
                x_train_m = x_train[sample]
            y_train_m = self.y_train[sample]
            if self.base_model is None:
                model = DecisionTreeClassifier(max_depth=1)
            else:
                model = self.base_model.set_params(**self.model_params)

            model.fit(x_train_m, y_train_m)
            self.models.append(model)


      
    def predict(self, x_test):
        y_test_hats = np.empty((len(self.models), len(x_test)))
        for i, model in enumerate(self.models):
            y_test_hats[i] = model.predict(x_test)
        
        return y_test_hats.mean(0)

    def calculate_accuracy(self, y_true, y_pred):
        y_pred_binary = (y_pred > 0.5).astype(int)  # Assuming binary classification
        return accuracy_score(y_true, y_pred_binary)
    def score(self, X, y):
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

bagger = Bagger(M=30, seed=123)
bagger.fit(x_train, y_train)
y_test_hat = bagger.predict(x_test)

accuracy = bagger.calculate_accuracy(y_test, y_test_hat)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7053874168027126


In [68]:
param_grid = {
    'base_model': [DecisionTreeClassifier(), LogisticRegression(), KNeighborsClassifier(), LinearRegression()],
    'M': [None,10,20,30,80, 100, 120],  # Number of weak learners
    'model_params': [
        {'max_depth': [1,2,3], 'criterion': ['gini', 'entropy']},  # Parameters for DecisionTreeClassifier
        {},  # Parameters for LogisticRegression
        {'n_neighbors': list(range(1, 20))},  # Parameters for KNeighborsClassifier
    ]
}
bagger = Bagger(M=30, seed=123)
scorer = make_scorer(accuracy_score)
grid_search = GridSearchCV(estimator=bagger, param_grid=param_grid, scoring=scorer, cv=5)
grid_search.fit(x_train, y_train)
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)


best_model = grid_search.best_estimator_
val_accuracy = best_model.score(x_val, y_val)
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))

# Test the best model on the test set
test_accuracy = best_model.score(x_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

Best Parameters:  {'M': 20, 'base_model': LogisticRegression(), 'model_params': {}}
Best Accuracy:  0.7154083374794695
Validation Accuracy: 71.28%
Test Accuracy: 71.53%


In [42]:

# Define the parameter distributions for RandomizedSearchCV
param_dist =  {
    'base_model': [None,DecisionTreeClassifier(), LogisticRegression(), KNeighborsClassifier(), LinearRegression()],
    'M': [None,10,20,30,80, 100, 120],  # Number of weak learners
    'model_params': [
        {'max_depth': [1,2,3], 'criterion': ['gini', 'entropy']},  # Parameters for DecisionTreeClassifier
        {},  # Parameters for LogisticRegression
        {'n_neighbors': list(range(1, 20))},  # Parameters for KNeighborsClassifier
        {}
    ]
}
bagger = Bagger(M=30, seed=123)
scorer = make_scorer(accuracy_score)
random_search = RandomizedSearchCV(estimator=bagger, param_distributions=param_dist, n_iter=18, scoring=scorer, cv=5, random_state=42)
random_search.fit(x_train, y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", random_search.best_params_)
print("Best Accuracy: ", random_search.best_score_)


best_model = random_search.best_estimator_
val_accuracy = best_model.score(x_val, y_val)
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))

# Test the best model on the test set
test_accuracy = best_model.score(x_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))


Best Parameters:  {'model_params': {}, 'base_model': LogisticRegression(), 'M': 10}
Best Accuracy:  0.7149329090393655
Validation Accuracy: 71.27%
Test Accuracy: 71.47%


## RandomForest

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class RandomForest:
    def __init__(self, M=100, max_depth=100, min_size=2, criterion='gini', seed=None):
        self.models = []  # List to store individual DecisionTree models
        self.M = M
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_size = min_size
        self.seed = seed
        self.training_errors = []
        self.prediction_errors = []

    def get_params(self, deep=True):
        return {'M': self.M, 'criterion': self.criterion, 'max_depth': self.max_depth, 'min_size': self.min_size}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.N, self.D = x_train.shape
        self.y_train = y_train.values if isinstance(y_train, pd.Series) else y_train

        seed = self.seed if self.seed is not None else np.random.randint(1, 1000)  # Set a random seed if None
        np.random.seed(seed)

        for _ in range(self.M):
            sample = np.random.choice(np.arange(self.N), size=self.N, replace=True)
            sample = sample.astype(int)

            if isinstance(x_train, pd.DataFrame):
                x_train_m = x_train.iloc[sample]
            else:
                x_train_m = x_train[sample]

            y_train_m = self.y_train[sample]

            model = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_size, criterion=self.criterion)
            model.fit(x_train_m, y_train_m)
            self.models.append(model)

        # Return the fitted model
        return self

    def predict(self, x_test):
        y_test_hats = np.empty((len(self.models), len(x_test)))
        for i, model in enumerate(self.models):
            y_test_hats[i] = model.predict(x_test)

        return y_test_hats.mean(0)

    def calculate_accuracy(self, y_true, y_pred):
        y_pred_binary = (y_pred > 0.5).astype(int)
        return accuracy_score(y_true, y_pred_binary)

# Example usage:
random_forest = RandomForest(M=30, max_depth=20, min_size=5, seed=123)
random_forest.fit(x_train, y_train)
y_test_hat = random_forest.predict(x_test)

accuracy = random_forest.calculate_accuracy(y_test, y_test_hat)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7294152120222697


In [70]:
param_grid = {
    'M': [10, 20, 30, 50],
    'max_depth': [10, 20, 30, 40],
    'min_size': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}


# Instantiate your RandomForest class
random_forest = RandomForest(seed=123)
scorer = make_scorer(random_forest.calculate_accuracy)
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=5, scoring=scorer)

grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Accuracy: ", best_score)

# Test the best model on the test set
best_model = grid_search.best_estimator_

y_test_hat = best_model.predict(x_test)
test_accuracy = best_model.calculate_accuracy(y_test, y_test_hat)
print(f"Test Accuracy: {test_accuracy}")

y_val_hat = best_model.predict(x_val)
val_accuracy = best_model.calculate_accuracy(y_val, y_val_hat)
print(f"Validation Accuracy: {val_accuracy}")


Best Parameters:  {'M': 50, 'criterion': 'entropy', 'max_depth': 10, 'min_size': 10}
Best Accuracy:  0.7407852886479235
Test Accuracy: 0.7395453974632676
Validation Accuracy: 0.7402461486939049


In [52]:
param_dist = {
    'M': [10, 20, 30, 50],  
    'max_depth': [10, 20, 30, 40],  
    'min_size': [2, 5, 10],  
    'criterion': ['gini', 'entropy']  
}

random_forest = RandomForest(seed=123)
scorer = make_scorer(random_forest.calculate_accuracy)
random_search = RandomizedSearchCV(estimator=random_forest, param_distributions=param_dist, n_iter=10, scoring=scorer, cv=5, random_state=42)

random_search.fit(x_train, y_train)

best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters: ", best_params)
print("Best Accuracy: ", best_score)

# Test the best model on the test set
best_model = random_search.best_estimator_

y_test_hat = best_model.predict(x_test)
test_accuracy = best_model.calculate_accuracy(y_test, y_test_hat)
print(f"Test Accuracy: {test_accuracy}")

y_val_hat = best_model.predict(x_val)
val_accuracy = best_model.calculate_accuracy(y_val, y_val_hat)
print(f"Validation Accuracy: {val_accuracy}")


Best Parameters:  {'min_size': 5, 'max_depth': 10, 'criterion': 'gini', 'M': 50}
Best Accuracy:  0.7407494033466662
Test Accuracy: 0.7402570220603625
Validation Accuracy: 0.7391577361018085


## XGBoost

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [12]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Instantiate XGBoost Classifier
xgb = XGBClassifier()
xgb.fit(x_train, y_train)

# Predict on the test set
y_pred = xgb.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')

Accuracy: 0.7379547071874084


## catboost

In [3]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [13]:
from catboost import CatBoostClassifier

# Convert categorical features to string type
x_train_cat = x_train.astype(str)
x_val_cat = x_val.astype(str)

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1,
    loss_function='Logloss'  # You can uncomment this line if you want to explicitly specify the loss function
)

cat_features = list(range(0, x_train.shape[1]))
clf.fit(x_train_cat, y_train, 
        cat_features=cat_features, 
        eval_set=(x_val_cat, y_val), 
        verbose=False
)


print(clf.get_params())

y_pred_test = clf.predict(x_test.astype(str))

# Calculate accuracy on the test set
accuracy_test = accuracy_score(y_test, y_pred_test)

print('CatBoost Test Accuracy:', accuracy_test)

{'iterations': 5, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
CatBoost Test Accuracy: 0.7351500690694462
